In [1]:
# =============================================================================
# Cell 1: Imports and Load Data
# =============================================================================
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import importlib

# Import terrain module (reload to get latest changes)
import terrain
importlib.reload(terrain)

# Load ERA5 data
era5_path = './era5vis-main/era5vis/data/era5_example_dataset.nc'
era5_full = xr.open_dataset(era5_path)

# Select Alps region
era5_data = era5_full.sel(
    latitude=slice(48, 45),
    longitude=slice(6, 16)
)

# Load terrain data (already cached)
terrain_ds = terrain.load_terrain_aspect_dataset("./terrain_cache/terrain_aspect_1km.nc")

print("✅ Data loaded successfully!")
print(f"\nERA5: {dict(era5_data.sizes)}")
print(f"Terrain: {terrain_ds.sizes['latitude']} × {terrain_ds.sizes['longitude']} @ {terrain_ds.attrs['resolution_m']}m")

Loaded terrain aspect dataset: 232 × 772
✅ Data loaded successfully!

ERA5: {'valid_time': 10, 'pressure_level': 5, 'latitude': 13, 'longitude': 41}
Terrain: 232 × 772 @ 990m


In [2]:
# =============================================================================
# Cell 2: Enhance ERA5 with Terrain Data
# =============================================================================

# Add terrain information to ERA5 data
era5_enhanced = terrain.enhance_era5_with_terrain(
    era5_data=era5_data,
    terrain_cache_dir="./terrain_cache",
    terrain_resolution_m=1000
)

print("\n✅ ERA5 enhanced with terrain!")
print("\nNew variables added:")
for var in ['terrain_elevation', 'terrain', 'geopotential_height', 'slope_aspect', 'perpendicular_wind', 'parallel_wind']:
    if var in era5_enhanced:
        print(f"  - {var}: {era5_enhanced[var].dims}")

Building terrain aspect dataset at ~1000m resolution...
Loading cached terrain from ./terrain_cache/srtm_alps_hr.tif
Loaded terrain: (7641, 25469), resolution ~30m
Downsampled: (7641, 25469) -> (232, 772), resolution ~990m
Saved to ./terrain_cache/terrain_aspect_1000m.nc
ERA5 domain: lat(45.0, 48.0), lon(6.0, 16.0)
Terrain elevation range on ERA5 grid: -32089m to 2788m

Terrain intersection summary:
   925 hPa: mean height ~  844m,   164/533 ( 30.8%) below terrain
   850 hPa: mean height ~ 1538m,    56/533 ( 10.5%) below terrain
   700 hPa: mean height ~ 3090m,     0/533 (  0.0%) below terrain
   500 hPa: mean height ~ 5664m,     0/533 (  0.0%) below terrain
   300 hPa: mean height ~ 9281m,     0/533 (  0.0%) below terrain

✅ ERA5 enhanced with terrain!

New variables added:
  - terrain_elevation: ('latitude', 'longitude')
  - terrain: ('valid_time', 'pressure_level', 'latitude', 'longitude')
  - geopotential_height: ('valid_time', 'pressure_level', 'latitude', 'longitude')
  - slope_a